# Bayesian A/B simulation

Suppose you have N consumers in a test panel choosing between options A and B. You want to infer from this data which option is really the most popular option.

This notebook explores the following question: can we save on A/B testing by using bayesian sequential testing - and how much?

The setup is very simple:
* generate some fake A/B choice data
* form a Beta prior that describes your belief about the probability of A being the more chosen option 
* with each step of M data points, update your posterior to reflect the data
* define your desired certainty level, and calculate the highest probability density interval (HPDI) of the posterior
* if HPDI is outside the region of practical equivalence (ROPE), discontinue testing and choose the more likely alternative
* if HPDI is completely inside ROPE, discontinue test and infer that A and B are equally good

In [ ]:
import numpy as np
import numpyro
import numpyro.distributions as dist
import jax.numpy as jnp
from jax import random
import matplotlib.pyplot as plt
import numpyro.optim as optim
from numpyro.infer import SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoLaplaceApproximation
import arviz as az
from typing import List
from collections import Counter
import pandas as pd
import seaborn as sns

import sys
sys.path.append("..")
from modules.faker import create_fake_ab_test_results
from modules.utils import f_symlog
from modules.bayes import sim_many_experiments, sim_single_experiment


## Example of one experiment

Generate fake A/B data with the faker module.

In [ ]:
n_resp = 1000
prob_a = 0.6
results = create_fake_ab_test_results(n=n_resp, prob_a=prob_a)

Inspect the results

In [ ]:
print(Counter(results["choice"]))
print("Percentage of A: {}".format(sum(results["choice"]==0)/len(results["choice"])))
print("Prob A was set at: {}".format(prob_a))

In this simple case of a binary choice experiment, we can solve the posterior analytically for the seen data:


In [ ]:
L = sum(results["choice"]) # number of B in data
W = len(results["choice"]) - L # number of A in data
print("Number of 0s (A) in data: {}".format(W))
print("Number of 0s (B) in data: {}".format(L))
x = jnp.linspace(0, 1, 101)
#jnp.exp(dist.Beta(W + 1, L + 1).log_prob(x))
plt.plot(x, jnp.exp(dist.Beta(W + 1, L + 1).log_prob(x)))

As we can see, the posterior shows that we can be quite sure that A is the more popular option.

To make this sensitive to prior beliefs, we can set a prior that describes our prior understanding. `Beta(X,Y)` prior describes prior data where A was chosen X times and B was chosen Y times. For example, if we have very strong opinion that A and B are equally good, we could use a prior such as `Beta(1000,1000)`. On the other hand, if we are unsure and want a uncommitted prior, we can use e.g. `Beta(2,2)`. We will use a `Beta(10,10)` prior as default in the model, to reflect the understanding that differences between A and B are unlikely to be huge.

In [ ]:
desired_certainty = 0.95
prior = dist.Beta(10,10)
#prior=dist.Uniform(0,1) # you could also use a flat prior, but that is quite unrealistic usually

In [ ]:
x = jnp.linspace(0, 1, 101)
plt.plot(x, jnp.exp(prior.log_prob(x)), "--")

As you can see, the prior `Beta(10,10)` shows that likely values are centered around 0.5, meaning A and B are equally good.

Generally, we can use the module `numpyro` to compute posteriors for our Beta model.

In [ ]:
def model(W, L, prior):
    p = numpyro.sample("p", prior)  # prior
    numpyro.sample("W", dist.Binomial(W + L, p), obs=W)  # binomial likelihood

L = 4
#print("Number of 0s (A) in data: {}".format(W))
W = 6
#print("Number of 0s (B) in data: {}".format(L))
#print("Distribution of seen data:{}".format(Counter(true_results["choice"][0:i])))
guide = AutoLaplaceApproximation(model)
svi = SVI(model, guide, optim.Adam(1), Trace_ELBO(), W=W, L=L, prior=prior)
params, losses = svi.run(random.PRNGKey(0), 10000, progress_bar=False)

# display summary of quadratic approximation
samples = guide.sample_posterior(random.PRNGKey(1), params, (10000,))
numpyro.diagnostics.print_summary(samples, prob=desired_certainty, group_by_chain=False)

Note that in this special beta-binomial case, we can also compute the posterior straight from the posterior distribution that we computed analytically!

In [ ]:
s_beta = dist.Beta(W+10, L+10).sample(random.PRNGKey(1), (10000,))
print("mean {:.2f}, std {:.2f}, median {:.2f}, 2.5% {:.2f}, 97.5% {:.2f}".format(
np.mean(s_beta), np.std(s_beta), np.median(s_beta), np.percentile(s_beta, 2.5), np.percentile(s_beta, 97.5)))

As we can see, results of the `numpyro` SVI computation and analytical solution are almost exactly the same (up to a certain level of randomness). Analytical solution - i.e. straight simulation from the posterior - is much faster, so we should use that whenever possible.

Now, let's define our function for analysis of the whole experiment. Inputs are:
* `desired_certainty`: level of certainty about the inference of A vs. B
* `true_results`: the results of the experiment
* `true_probs_a`: The true probability of theta for generation of the data.
* `min_n_obs`: Minimum number of observations that must be run before breaking the experiment.
* `inference_strategy`: `"analytical"` for using the analytical solution, `"svi"` for SVI. Analytical is much faster.
* `rope_width`: Radius of the ROPE region.
* `null_effect`: Value of no effect (0.5 in binary choice).
* `step`: Evaluate breaking of experiment at each `step` obs. Smaller is more accurate, but makes analysis runtime longer.


In [ ]:
n_resp = 500
prob_a = 0.60
certainty_level = 0.90
results = faker.create_fake_ab_test_results(n=n_resp, prob_a=prob_a)
res = sim_single_experiment(certainty_level, results, true_probs_a=[prob_a], step=50, print_progress=True)

In [ ]:
res

In [ ]:
cutoff = (res.ended==True).idxmax()
res.iloc[(cutoff-1):(cutoff + 5)]
cutoff_n = res.loc[(res.ended==True).idxmax(),"n"]

Plot how the CI of theta has evolved over the experiment.

In [ ]:
res.hvplot.line(x="n", 
                y=["hpdi_lower","hpdi_upper","true_prob_a"]) * \
    hv.HLine(0.5).opts(
    opts.HLine(color='blue', line_width=1, line_dash="dashed")) * \
    hv.VLine(cutoff_n).opts(
    opts.VLine(color="red", line_width=0.8, line_dash="dashed"))

In [ ]:
d = pd.melt(res, id_vars=["n"], value_vars=["hpdi_lower","hpdi_upper","true_prob_a"])
fig = sns.lineplot(data=d, x="n",y="value", hue="variable")
plt.axhline(0.5,linestyle='--', color="blue")
plt.axvline(cutoff_n, linestyle="--", color="red")
plt.show()

## Simulating several experiments

Extension of one experiment to several is simple. We just define a grid of parameters, run N experiments for each cell in grid, and collect the results to see how our bayesian system behaves at each parameter value.


Now let us run our simulation for the parameter grid. Below you will see the defined parameters, and the setting of `max_n_resp`, describing maximum number of responses per experiment. The higher this number, the more chances our bayesian analysis has to break early and save on cost.

In [ ]:
max_n_resp = 500
test_output = False
if test_output:
    all_results = sim_many_experiments(2, true_probs_a=[0.55, 0.65], desired_cert_list=[0.8, 0.9], n_resp=max_n_resp)
else:
    all_results = sim_many_experiments(1000, true_probs_a=[0.52, 0.53, 0.54, 0.55, 0.60, 0.65], 
                                       desired_cert_list=[0.8, 0.9, 0.95, 0.99], n_resp=max_n_resp)

In [ ]:
all_results

Make a DF that describes the result of each experiment at when it ended (whether due to bayesian cutoff or end of data).

In [ ]:
all_results.loc[all_results.stopping_reason=="end_of_experiment","conclusion"] = "uncertain"

In [ ]:
cutoff_results = all_results[all_results["ended"]==True].dropna(axis=0, how="any").groupby("exp_no").head(1)

In [ ]:
cutoff_results

In [ ]:
cutoff_results.groupby(["desired_certainty","true_prob_a"])["conclusion"].value_counts(dropna=False)

In [ ]:
del all_results

In [ ]:
for cert in sorted(cutoff_results.desired_certainty.unique()):
    display(cutoff_results[cutoff_results.desired_certainty == cert].groupby(["desired_certainty","true_prob_a"])["conclusion"].value_counts(dropna=False).reset_index(name="values").\
        pivot(index=["desired_certainty","true_prob_a"], columns=["conclusion"],values="values").reset_index().\
        hvplot(kind="bar", x="true_prob_a", y=["A","B","uncertain"], 
               stacked=True, subplots=False, rot=90, title="Certainty: {}".format(str(cert))))


In [ ]:
from matplotlib import rcParams
rcParams['figure.figsize'] = 4,2
for cert in sorted(cutoff_results.desired_certainty.unique()):
    
    d = cutoff_results[cutoff_results.desired_certainty == cert].\
        loc[:,["true_prob_a","conclusion"]]
        #   id_vars=["true_prob_a"], value_vars=["A","B","uncertain"]).fillna(0)
    #display(d)
    
    sns.histplot(data=d, x="true_prob_a",hue="conclusion",multiple="stack")
    plt.title("Certainty level: {}".format(cert))
    plt.show()

The bars show us immediately a clear trend: as true probability increases, the number of `uncertain` conclusions decreases. This is intuitive - the stronger the true preference for A, the easier it is to detect with a fixed N of data.

What about mistakes? Leaving out the uncertain conclusions, how often does our system make an erroneous bayesian inference?

In [ ]:
d = cutoff_results[cutoff_results.conclusion != "uncertain"].groupby(["desired_certainty","true_prob_a"])["correct"].value_counts(dropna=False, normalize=True).reset_index(name="value").fillna(0)
d=d.loc[d.correct==True,["desired_certainty","true_prob_a","value"]]\
    .pivot("desired_certainty","true_prob_a","value").sort_values("desired_certainty",ascending=False)
display(d.head(10))
sns.heatmap(d, linewidths=0.5, cmap="PuOr", center=0.95)

In the figure above, we see that all versions of the model are over 97% accurate, and if true probability is over 55% then they are perfectly accurate.

To make a business decision on whether and how to use the system, we can analyse the impact on costs. This analysis assumes a fixed cost per response, and a very high cost on an erroneous inference between A and B. Naturally, such costs differ between use cases, and should preferably be elicited either from historical records or SMEs.

In [ ]:
cost_per_response = 1
cost_of_wrong_ab_decision = 10**6 # million
print("Cost per response {}, cost of wrong A/B decision {}\n".format(cost_per_response, cost_of_wrong_ab_decision))
params = set(zip(cutoff_results.desired_certainty.values, cutoff_results.true_prob_a.values))
params = list(sorted(params))
#print(list(sorted(params)))

cost_outcome_df = pd.DataFrame(params)#.pivot(index=0, columns=1, values=2)
cost_outcome_df.columns = ["desired_certainty","true_prob_a"]
cost_outcome_df[["saved_sampling","wrong_decisions","cost_wrong_decisions","mistake_prob","total"]] = np.nan

for cert, true_prob in params:
    print("For certainty {}, prob_a {} we got:".format(cert, true_prob))
    d = cutoff_results.loc[(cutoff_results.desired_certainty == cert) & (cutoff_results.true_prob_a == true_prob) & \
                          (cutoff_results.conclusion != "uncertain")]
    uncertain_results = len(cutoff_results.loc[(cutoff_results.desired_certainty == cert) & (cutoff_results.true_prob_a == true_prob) & \
                          (cutoff_results.conclusion == "uncertain")])
    
    max_sample = max_n_resp
    saved_in_sampling = sum([max_sample-x for x in d.n])
    n_wrong_decisions = sum(d.correct == False)
    correct_decisions = sum(d.correct == True)
    wrong_decision_cost = cost_of_wrong_ab_decision * n_wrong_decisions
    mistake_prob = (n_wrong_decisions) / (len(d) + uncertain_results)
    print("Saved money in sampling {:d} (max sample {})".format(saved_in_sampling, max_sample))
    print("Lost in wrong_decisions {:d}, probability of mistake {:.2%}".format(wrong_decision_cost, mistake_prob))
    print("Total {:+d}\n".format(saved_in_sampling - wrong_decision_cost))
    
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob), 
                        "saved_sampling"]=saved_in_sampling
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob), 
                        "uncertain_results"]=uncertain_results
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob),
                        "wrong_decisions"]=n_wrong_decisions
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob),
                        "correct_decisions"]=correct_decisions
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob),
                        "cost_wrong_decisions"]=wrong_decision_cost
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob),
                        "mistake_prob"]=mistake_prob
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob),
                        "total"] = saved_in_sampling - wrong_decision_cost
    cost_outcome_df.loc[(cost_outcome_df.desired_certainty == cert) & (cost_outcome_df.true_prob_a == true_prob),
                        "save_per_exp"] = (saved_in_sampling - wrong_decision_cost) / len(d)
    
    cost_outcome_df = cost_outcome_df[["desired_certainty","true_prob_a","uncertain_results", "correct_decisions",
                              "wrong_decisions","saved_sampling","cost_wrong_decisions","mistake_prob",
                              "total","save_per_exp"]]

In [ ]:
display(cost_outcome_df)

What do the numbers tell us? Well, for example with a true difference of 0.55 and certainty level 0.99, we would expect almost 250 euros benefit per each test that was run. For 1000 tests, such a model broke early 993 times, was uncertain 7 times and made an incorrect inference 0 times!

In [ ]:
sns.heatmap(cost_outcome_df.pivot("desired_certainty","true_prob_a","total").\
            sort_values("desired_certainty",ascending=False),
            linewidths=0.5, cmap="PuOr",center=0)

This last figure plots average cost/saving per experiment according to true probability of A being better, for various certainty levels. What becomes clear is that the system saves costs when A is in prior probable to be better.

In [ ]:
symlog_costs=cost_outcome_df[["desired_certainty","true_prob_a","total","save_per_exp"]]

symlog_costs.loc[:,["log_total"]] = symlog_costs.total.apply(f_symlog)
symlog_costs.loc[:,["log_save_per_exp"]] = symlog_costs.save_per_exp.apply(f_symlog)

d = pd.melt(symlog_costs, id_vars=["desired_certainty","true_prob_a"],value_vars="log_save_per_exp")
display(d.head(5))
rcParams['figure.figsize'] = 6,3
palette = sns.color_palette("mako_r", d.desired_certainty.nunique())
g_results=sns.lineplot(data=d, x="true_prob_a",y="value",hue="desired_certainty", palette=palette)
#g_results.set(yscale='log')
plt.axhline(0, linestyle="--", color="blue")
#g_results.set(xticks=sample_count)
#g_results.set(xticklabels=sample_count)

Based on the cost and benefit analysis, using the bayesian model makes sense if the true probability is at least 55%. When the true probability is less, then the system makes mistakes in inference, resulting in big losses as per the assumed large cost of erroneous inference.